In [0]:
from pyspark.sql.functions import count, countDistinct,col
from pyspark.sql.functions import sum as spark_sum

### Loading the race_results parquet file

In [0]:
race_results = spark.read.parquet('/mnt/blobstorage/race_results').withColumn('points',col('points').cast('int'))
display(race_results)

In [0]:
race_results.printSchema()

In [0]:
race_results.select(count("race_name")).show()

In [0]:
race_results.select(countDistinct("contructor_ref")).show()

In [0]:
race_results.select(spark_sum("points")).show()


In [0]:
race_results.filter("driver_name= 'Lewis Hamilton'").select(spark_sum("points")).show()

In [0]:
race_results.filter("driver_name= 'Max Verstappen'").select(spark_sum("points")).show()

In [0]:
race_results.filter("contructor_ref= 'mercedes'").select(spark_sum("points")).show()

### Group by

### For driver Lewis Hamliton we will take count of different races and sum the points

In [0]:
race_results.filter("driver_name= 'Lewis Hamilton'").select(spark_sum("points"),\
count("race_name")).withColumnRenamed('sum(points)','total_points')\
.withColumnRenamed('count(DISTINCT race_name)','number_of_races').show()

### For driver Max Verstappen we will take count of different races and sum the points

In [0]:
race_results.filter("driver_name= 'Max Verstappen'").select(spark_sum("points"),\
count("race_name")).withColumnRenamed('sum(points)','total_points')\
.withColumnRenamed('count(DISTINCT race_name)','number_of_races').show()

In [0]:
race_results.groupBy("driver_name").sum("points").withColumnRenamed('sum(points)','total_points').show()

In [0]:
race_results.groupBy("driver_name").avg("points").withColumnRenamed('sum(points)','total_points').show()

In [0]:
race_results\
    .groupBy("driver_name")\
        .agg(spark_sum("points").alias("total_points"),
             count("race_name").alias("number_of_races")).show()
race_results\
    .groupBy("driver_name")

### Window functions

In [0]:
demo_grouped_df = race_results\
    .groupBy("race_year","driver_name")\
        .agg(spark_sum("points").alias("total_points"),
             count("race_name").alias("number_of_races"))
demo_grouped_df.show()

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank
driver_rank = Window.partitionBy("race_year").orderBy(desc("total_points"))
d_rank = demo_grouped_df.withColumn("rank",rank().over(driver_rank))
d_rank.show()

### Using positions to assign rank based on total points and number of wins

In [0]:
from pyspark.sql.functions import when

In [0]:
display(race_results)

In [0]:
driver_standing_df = race_results\
    .groupBy("race_year","driver_name","driver_nationality")\
        .agg(spark_sum("points").alias("total_points"),
             count(when(col("position")==1, True)).alias("wins"))
driver_standing_df.show()

In [0]:
display(driver_standing_df.filter("race_year=2020"))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank, asc
driver_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"),desc("wins"))
final_df = driver_standing_df.withColumn("rank",rank().over(driver_rank_spec))
final_df.show()


In [0]:
display(final_df.filter("race_year=2020"))

### Wrting the file in parquet file

In [0]:
final_df.write.mode("overwrite").parquet("/mnt/blobstorage/drivers_championship_standing")